In [1]:
%pip install --quiet langchain langchain-core langchain-community


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Define MariTalk API key and LLM model
MARITALK_API_KEY = os.getenv('MARITALK_API_KEY')
MARITALK_LLM_MODEL = "sabia-3"

# Define Tavily API key
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

# Construct React Agent

In [3]:
%pip install --quiet tavily-python 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.chat_models import ChatMaritalk

tools = [TavilySearchResults(max_results=1)]

# Get the prompt from langchain hub
prompt = hub.pull("hwchase17/react")

llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

# Construct the React agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [5]:
agent_executor.invoke({"input": "What is Langchain?"})



> Entering new AgentExecutor chain...
 Question: What is Langchain?
Thought: To provide an accurate and comprehensive answer, I should first gather information about Langchain.
Action: tavily_search_results_json
Action Input: "What is Langchain?"[{'url': 'https://www.ibm.com/think/topics/langchain', 'content': 'What Is LangChain? | IBM\nWhat is LangChain?\nArtificial Intelligence\n31 October 2023\nLink copied\nWhat is LangChain?\nLangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Available in both Python- and Javascript-based libraries, LangChain’s tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents. [...] LangChain is essentially a library of abstractions for Python and Javascript, representing common steps and concepts necessary to work with language models. These modular components—like functions and object classes—serve as the building blocks of generative

{'input': 'What is Langchain?',
 'output': 'LangChain is an open source orchestration framework designed for the development of applications using large language models (LLMs). It provides tools and APIs in both Python and Javascript, making it easier to build applications driven by LLMs such as chatbots and virtual agents. LangChain simplifies the process by offering a library of abstractions for common steps and concepts needed when working with language models. Its modular components, such as functions and object classes, act as building blocks for generative AI programs. These components can be "chained" together to create applications, thereby reducing the amount of code and deep understanding required to perform complex natural language processing (NLP) tasks. Although LangChain\'s abstracted approach may somewhat limit the control an expert has over the finer details, it significantly lowers the barrier of entry for developing AI applications.'}

# Message Persistence

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_community.chat_models import ChatMaritalk

# Define the Maritalk model
model = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

# Define the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant!"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
print(prompt_template)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

input_variables=['messages'] input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.Funct

In [7]:
# Define the configuration
config = {"configurable": {"thread_id": "thread_id_1"}}
query = "Hi! I'm an undergraduate computer science student."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello! It's great to meet you. How can I assist you today?


In [8]:
# Define the query
query = "What do I major in?"

# Invoke the agent
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a computer science student, your major would be in Computer Science. This typically covers a broad range of topics including programming, algorithms, data structures, computer architecture, and software engineering.


# Managing Conversation History

In [9]:
%pip install --quiet trimmer transformers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages
from langchain_community.chat_models import ChatMaritalk

# Define the model
model = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

# Define trimmer
trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# Define the messages
messages = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="Hi there! I'm an undergraduate computer science student"),
    AIMessage(content="Hello! How can I help you today?"),
    HumanMessage(content="I am trying to learn about data structures and algorithms"),
    AIMessage(content="Data structures and algorithms are a fundamental part of computer science!"),
]

trimmer.invoke(messages)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Hi there! I'm an undergraduate computer science student", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I am trying to learn about data structures and algorithms', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Data structures and algorithms are a fundamental part of computer science!', additional_kwargs={}, response_metadata={})]

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
    ]
)
print(prompt_template)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

input_variables=['messages'] input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.Funct

In [12]:
# Define the configuration
config = {"configurable": {"thread_id": "thread_id_2"}}
query = "What was my previous interaction? What computer science concepts did I mention?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You mentioned that you are an undergraduate computer science student and that you're trying to learn about data structures and algorithms.
